<a href="https://colab.research.google.com/github/nguyenvuong1309/Do-an-lap-trinh-an-toan/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
import pandas as pd
from numpy import zeros, newaxis
import numpy as np
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D



In [3]:
csv_file_path = '/content/drive/MyDrive/Colab/dataset/sample.csv'
df = pd.read_csv(csv_file_path)


In [4]:
print(df.shape)

(181641, 52)


In [5]:
print(df[:1])

   Index         1         2         3         4         5         6  \
0      0  0.252445 -0.119424 -0.035009 -0.059346 -0.057337  0.145114   

          7         8         9  ...        42        43        44        45  \
0 -0.249521  0.231396 -0.202908  ...  0.188895  0.113198 -0.120779  0.115096   

         46        47        48        49        50  label  
0 -0.233971 -0.074264 -0.472756 -0.001527 -0.665612    0.0  

[1 rows x 52 columns]


In [6]:
dataset = df

In [7]:
dataset = tf.constant(dataset)
dataset = dataset[1:,1:]

In [8]:
def splits(dataset,TRAIN_RATIO,VAL_RATIO,TEST_RATIO):
  DATASET_SIZE = len(dataset)

  train_dataset = dataset.take(int(DATASET_SIZE * TRAIN_RATIO))

  val_test_dataset = dataset.skip(int(DATASET_SIZE * TRAIN_RATIO))

  val_dataset = val_test_dataset.take(int(DATASET_SIZE * VAL_RATIO))

  test_dataset = val_test_dataset.skip(int(DATASET_SIZE * VAL_RATIO))

  return train_dataset, val_dataset, test_dataset

In [9]:
len(dataset)

181640

In [10]:
TRAIN_RATIO=0.6
VAL_RATIO=0.2
TEST_RATIO=0.2

#dataset = tf.data.Dataset.range(10)
#train_dataset, val_dataset, test_dataset = splits(dataset,TRAIN_RATIO,VAL_RATIO,TEST_RATIO)
train_dataset = dataset[:int(TRAIN_RATIO * len(dataset))]
val_dataset = dataset[int(TRAIN_RATIO * len(dataset)) : int(TRAIN_RATIO * len(dataset) + VAL_RATIO * len(dataset))]
test_dataset = dataset[(int(TRAIN_RATIO * len(dataset)) + int(VAL_RATIO * len(dataset))) :]

In [11]:
print(train_dataset.shape)
print(val_dataset.shape)
print(test_dataset.shape)

(108984, 51)
(36328, 51)
(36328, 51)


In [12]:
108981 + 36327 * 2

181635

In [13]:
X_train = train_dataset[:,:-1]
y_train = train_dataset[:,-1]
print(X_train.shape)
print(y_train.shape)

(108984, 50)
(108984,)


In [14]:
X_train = tf.expand_dims(X_train, axis=1)

In [15]:
print(X_train.shape)

(108984, 1, 50)


In [16]:
y_train = tf.keras.utils.to_categorical(y_train, num_classes=41)
y_train.shape

(108984, 41)

In [17]:
X_test = test_dataset[:,:-1]
y_test = test_dataset[:,-1]
print(X_test.shape)
print(y_test.shape)
X_test = tf.expand_dims(X_test, axis=1)
print(X_test.shape)

(36328, 50)
(36328,)
(36328, 1, 50)


In [18]:
y_test = tf.keras.utils.to_categorical(y_test, num_classes=41)
y_test.shape

(36328, 41)

# Sequential Model

In [19]:
model = tf.keras.models.Sequential([
     tf.keras.layers.InputLayer(input_shape=(1,50)),

    # tf.keras.layers.Conv2D(32, (3, 3), padding='same'),
    # tf.keras.layers.Activation('relu'),
    # tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1000),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dense(512),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dense(41),
    tf.keras.layers.Activation('softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 50)                0         
                                                                 
 dense (Dense)               (None, 1000)              51000     
                                                                 
 activation (Activation)     (None, 1000)              0         
                                                                 
 dense_1 (Dense)             (None, 512)               512512    
                                                                 
 activation_1 (Activation)   (None, 512)               0         
                                                                 
 dense_2 (Dense)             (None, 41)                21033     
                                                                 
 activation_2 (Activation)   (None, 41)                0

In [20]:
 opt = tf.keras.optimizers.RMSprop(learning_rate=0.0001)
 model.compile(loss='categorical_crossentropy',
                optimizer=opt,
                metrics=['accuracy'],
                #run_eagerly=True
               )

In [21]:
model.fit(X_train,y_train,epochs=1)

Epoch 1/3
3406/3406 [==============================] - 47s 14ms/step - loss: 1.0192 - accuracy: 0.7626
Epoch 2/3
3406/3406 [==============================] - 42s 12ms/step - loss: 0.8748 - accuracy: 0.7823
Epoch 3/3
3406/3406 [==============================] - 47s 14ms/step - loss: 0.7980 - accuracy: 0.7942


In [22]:
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)

print('\nTest accuracy:', test_acc)

1136/1136 - 4s - loss: 0.7819 - accuracy: 0.7858 - 4s/epoch - 3ms/step

Test accuracy: 0.7858400940895081


# **LSTM Model**

In [28]:
n_most_common_words = 8000
emb_dim = 128

In [29]:
model_lstm = tf.keras.models.Sequential()
model_lstm.add(Embedding(n_most_common_words, emb_dim, input_length=50))
model_lstm.add(SpatialDropout1D(0.7))
model_lstm.add(LSTM(64, dropout=0.7, recurrent_dropout=0.7))
model_lstm.add(Dense(41, activation='softmax'))
model_lstm.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model_lstm.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 50, 128)           1024000   
                                                                 
 spatial_dropout1d_1 (Spati  (None, 50, 128)           0         
 alDropout1D)                                                    
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dense_4 (Dense)             (None, 41)                2665      
                                                                 
Total params: 1076073 (4.10 MB)
Trainable params: 1076073 (4.10 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [30]:
model_lstm.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'],run_eagerly=True)

In [31]:
accr = model_lstm.evaluate(tf.squeeze(tf.abs(X_test)),y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

1136/1136 [==============================] - 399s 351ms/step - loss: 3.6938 - acc: 9.9097e-04
Test set
  Loss: 3.694
  Accuracy: 0.001


In [ ]:
model_lstm.fit(tf.squeeze(tf.abs(X_train)), y_train, epochs=1)

 903/3406 [======>.......................] - ETA: 34:51 - loss: 1.1505 - acc: 0.7551